In [1]:
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as func
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,FloatType

In [5]:



spark = SparkSession.builder.appName("Broadcast").getOrCreate()

states = {"NY":"New York","CA":"California"}
broadcastStates = spark.sparkContext.broadcast(states)

data = [("James","Smith","USA","CA"),
        ("Michale","Rose","USA","NY")]

columns = ["firstname","lastname","country","state"]

df = spark.createDataFrame(data=data,schema=columns)
df.printSchema()
df.show(truncate=True)

def state_convert(code):
    return broadcastStates.value[code]

result = df.rdd.map(lambda x: (x[0],x[1],x[2],state_convert(x[3]))).toDF(columns)
result.show(truncate=False)
spark.stop()

root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)

+---------+--------+-------+-----+
|firstname|lastname|country|state|
+---------+--------+-------+-----+
|    James|   Smith|    USA|   CA|
|  Michale|    Rose|    USA|   NY|
+---------+--------+-------+-----+

+---------+--------+-------+----------+
|firstname|lastname|country|state     |
+---------+--------+-------+----------+
|James    |Smith   |USA    |California|
|Michale  |Rose    |USA    |New York  |
+---------+--------+-------+----------+



In [4]:
spark.stop()

In [5]:
### rdd
conf = SparkConf().setAppName("Spark Demo").setMaster("local")
sc = SparkContext(conf=conf)
data = sc.parallelize([(1, 2), (3, 4), (3, 6)])

In [9]:
group = data.mapValues(lambda x:(x,1)).reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))

In [10]:
for x,y in group.collect():
    print(f"key: {x}, value:{y}")

key: 1, value:(2, 1)
key: 3, value:(10, 2)


In [11]:
counts = group.count()
print(f"el numero de elementos es {counts}")

el numero de elementos es 2


In [16]:
filters = group.filter(lambda x:x[0]>2)
for i in filters.collect():
    print(i)

(3, (10, 2))


In [17]:
from operator import add
adding = data.reduce(add)
print(f"adding all elementes {adding}")

adding all elementes (1, 2, 3, 4, 3, 6)


In [18]:
sc.stop()